In [1]:
import torch
from torch import nn
import utils

In [2]:
def conv_block(input_channels, num_channels):
    return nn.Sequential(
        nn.BatchNorm2d(input_channels), 
        nn.ReLU(),
        nn.Conv2d(input_channels, num_channels, kernel_size=3, padding=1)
    )

In [4]:
class DenseBlock(nn.Module):
    """
        稠密块
    """
    def __init__(self, num_convs, input_channels, num_channels):
        super(DenseBlock, self).__init__()
        layer = []
        for i in range(num_convs):
            layer.append(conv_block(num_channels * i + input_channels, num_channels))
        self.net = nn.Sequential(*layer)

    def forward(self, X):
        for blk in self.net:
            Y = blk(X)
            # 连接通道维度上每个块的输入和输出
            X = torch.cat((X, Y), dim=1)
        return X

In [5]:
blk = DenseBlock(2, 3, 10)
X = torch.randn(4, 3, 8, 8)
Y = blk(X)
Y.shape

torch.Size([4, 23, 8, 8])

In [6]:
def transition_block(input_channels, num_channels):
    """
        过渡层
        （通过1*1卷积核和池化来降低复杂度）
    """
    return nn.Sequential(
        nn.BatchNorm2d(input_channels), 
        nn.ReLU(),
        nn.Conv2d(input_channels, num_channels, kernel_size=1),
        nn.AvgPool2d(kernel_size=2, stride=2)
    )

In [7]:
blk = transition_block(23, 10)
blk(Y).shape

torch.Size([4, 10, 4, 4])

In [8]:
b1 = nn.Sequential(
    nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
    nn.BatchNorm2d(64), 
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
)

In [9]:
# num_channels为当前的通道数
num_channels, growth_rate = 64, 32
num_convs_in_dense_blocks = [4, 4, 4, 4]
blks = []
for i, num_convs in enumerate(num_convs_in_dense_blocks):
    blks.append(DenseBlock(num_convs, num_channels, growth_rate))
    # 上一个稠密块的输出通道数
    num_channels += num_convs * growth_rate
    # 在稠密块之间添加一个转换层，使通道数量减半
    if i != len(num_convs_in_dense_blocks) - 1:
        blks.append(transition_block(num_channels, num_channels // 2))
        num_channels = num_channels // 2

In [10]:
net = nn.Sequential(
    b1, 
    *blks,
    nn.BatchNorm2d(num_channels), 
    nn.ReLU(),
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(),
    nn.Linear(num_channels, 10)
)

In [11]:
lr, num_epochs, batch_size = 0.1, 10, 128
train_iter, test_iter = utils.load_data_fashion_mnist(batch_size, resize=96)
utils.train_ch6(net, train_iter, test_iter, num_epochs, lr, utils.try_gpu())

training on: [cuda:0], [2024-06-07 03:01:57]
epoch: 1/10, loss 0.456, train acc 0.836, test acc 0.805
epoch: 2/10, loss 0.280, train acc 0.897, test acc 0.857
epoch: 3/10, loss 0.238, train acc 0.913, test acc 0.893
epoch: 4/10, loss 0.210, train acc 0.923, test acc 0.908
epoch: 5/10, loss 0.190, train acc 0.931, test acc 0.906
epoch: 6/10, loss 0.175, train acc 0.935, test acc 0.903
epoch: 7/10, loss 0.162, train acc 0.940, test acc 0.909
epoch: 8/10, loss 0.148, train acc 0.945, test acc 0.922
epoch: 9/10, loss 0.140, train acc 0.948, test acc 0.866
epoch: 10/10, loss 0.127, train acc 0.953, test acc 0.889
*** 1016.0 examples/sec on cuda:0 - [0:00:09:50], all: [0:00:11:44] ***
